## First Innings Score Prediction

In [1]:
# Importing essential libraries
import pandas as pd
import pickle

# Loading the dataset
df = pd.read_csv('ipl.csv')

In [3]:
df.head()

mid        date                  venue               bat_team  \
0    1  2008-04-18  M Chinnaswamy Stadium  Kolkata Knight Riders   
1    1  2008-04-18  M Chinnaswamy Stadium  Kolkata Knight Riders   
2    1  2008-04-18  M Chinnaswamy Stadium  Kolkata Knight Riders   
3    1  2008-04-18  M Chinnaswamy Stadium  Kolkata Knight Riders   
4    1  2008-04-18  M Chinnaswamy Stadium  Kolkata Knight Riders   

                     bowl_team      batsman   bowler  runs  wickets  overs  \
0  Royal Challengers Bangalore   SC Ganguly  P Kumar     1        0    0.1   
1  Royal Challengers Bangalore  BB McCullum  P Kumar     1        0    0.2   
2  Royal Challengers Bangalore  BB McCullum  P Kumar     2        0    0.2   
3  Royal Challengers Bangalore  BB McCullum  P Kumar     2        0    0.3   
4  Royal Challengers Bangalore  BB McCullum  P Kumar     2        0    0.4   

   runs_last_5  wickets_last_5  striker  non-striker  total  
0            1               0        0            0    222  
1            1               0        0            0    222  
2            2               0        0            0    222  
3            2               0        0            0    222  
4            2               0        0            0    222

In [4]:
# --- Data Cleaning ---
# Removing unwanted columns
columns_to_remove = ['mid', 'venue', 'batsman', 'bowler', 'striker', 'non-striker']
df.drop(labels=columns_to_remove, axis=1, inplace=True)

In [5]:
df.head()

date               bat_team                    bowl_team  runs  \
0  2008-04-18  Kolkata Knight Riders  Royal Challengers Bangalore     1   
1  2008-04-18  Kolkata Knight Riders  Royal Challengers Bangalore     1   
2  2008-04-18  Kolkata Knight Riders  Royal Challengers Bangalore     2   
3  2008-04-18  Kolkata Knight Riders  Royal Challengers Bangalore     2   
4  2008-04-18  Kolkata Knight Riders  Royal Challengers Bangalore     2   

   wickets  overs  runs_last_5  wickets_last_5  total  
0        0    0.1            1               0    222  
1        0    0.2            1               0    222  
2        0    0.2            2               0    222  
3        0    0.3            2               0    222  
4        0    0.4            2               0    222

In [6]:
df['bat_team'].unique()

array(['Kolkata Knight Riders', 'Chennai Super Kings', 'Rajasthan Royals',
       'Mumbai Indians', 'Deccan Chargers', 'Kings XI Punjab',
       'Royal Challengers Bangalore', 'Delhi Daredevils',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Sunrisers Hyderabad',
       'Rising Pune Supergiants', 'Gujarat Lions',
       'Rising Pune Supergiant'], dtype=object)

In [7]:
# Keeping only consistent teams
consistent_teams = ['Kolkata Knight Riders', 'Chennai Super Kings', 'Rajasthan Royals',
                    'Mumbai Indians', 'Kings XI Punjab', 'Royal Challengers Bangalore',
                    'Delhi Daredevils', 'Sunrisers Hyderabad']

In [8]:
df = df[(df['bat_team'].isin(consistent_teams)) & (df['bowl_team'].isin(consistent_teams))]

In [9]:
# Removing the first 5 overs data in every match
df = df[df['overs']>=5.0]

In [10]:
df.head()

date               bat_team                    bowl_team  runs  \
32  2008-04-18  Kolkata Knight Riders  Royal Challengers Bangalore    61   
33  2008-04-18  Kolkata Knight Riders  Royal Challengers Bangalore    61   
34  2008-04-18  Kolkata Knight Riders  Royal Challengers Bangalore    61   
35  2008-04-18  Kolkata Knight Riders  Royal Challengers Bangalore    61   
36  2008-04-18  Kolkata Knight Riders  Royal Challengers Bangalore    61   

    wickets  overs  runs_last_5  wickets_last_5  total  
32        0    5.1           59               0    222  
33        1    5.2           59               1    222  
34        1    5.3           59               1    222  
35        1    5.4           59               1    222  
36        1    5.5           58               1    222

In [11]:
print(df['bat_team'].unique())
print(df['bowl_team'].unique())

['Kolkata Knight Riders' 'Chennai Super Kings' 'Rajasthan Royals'
 'Mumbai Indians' 'Kings XI Punjab' 'Royal Challengers Bangalore'
 'Delhi Daredevils' 'Sunrisers Hyderabad']
['Royal Challengers Bangalore' 'Kings XI Punjab' 'Delhi Daredevils'
 'Rajasthan Royals' 'Mumbai Indians' 'Chennai Super Kings'
 'Kolkata Knight Riders' 'Sunrisers Hyderabad']


In [12]:
# Converting the column 'date' from string into datetime object
from datetime import datetime
df['date'] = df['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

In [13]:
df

date               bat_team                    bowl_team  runs  \
32    2008-04-18  Kolkata Knight Riders  Royal Challengers Bangalore    61   
33    2008-04-18  Kolkata Knight Riders  Royal Challengers Bangalore    61   
34    2008-04-18  Kolkata Knight Riders  Royal Challengers Bangalore    61   
35    2008-04-18  Kolkata Knight Riders  Royal Challengers Bangalore    61   
36    2008-04-18  Kolkata Knight Riders  Royal Challengers Bangalore    61   
...          ...                    ...                          ...   ...   
75884 2017-05-19  Kolkata Knight Riders               Mumbai Indians   106   
75885 2017-05-19  Kolkata Knight Riders               Mumbai Indians   107   
75886 2017-05-19  Kolkata Knight Riders               Mumbai Indians   107   
75887 2017-05-19  Kolkata Knight Riders               Mumbai Indians   107   
75888 2017-05-19  Kolkata Knight Riders               Mumbai Indians   107   

       wickets  overs  runs_last_5  wickets_last_5  total  
32           0    5.1           59               0    222  
33           1    5.2           59               1    222  
34           1    5.3           59               1    222  
35           1    5.4           59               1    222  
36           1    5.5           58               1    222  
...        ...    ...          ...             ...    ...  
75884        9   18.1           29               4    107  
75885        9   18.2           29               4    107  
75886        9   18.3           28               4    107  
75887        9   18.4           24               4    107  
75888       10   18.5           23               5    107  

[40108 rows x 9 columns]

In [14]:
# --- Data Preprocessing ---
# Converting categorical features using OneHotEncoding method
encoded_df = pd.get_dummies(data=df, columns=['bat_team', 'bowl_team'])

In [15]:
encoded_df.head()

date  runs  wickets  overs  runs_last_5  wickets_last_5  total  \
32 2008-04-18    61        0    5.1           59               0    222   
33 2008-04-18    61        1    5.2           59               1    222   
34 2008-04-18    61        1    5.3           59               1    222   
35 2008-04-18    61        1    5.4           59               1    222   
36 2008-04-18    61        1    5.5           58               1    222   

    bat_team_Chennai Super Kings  bat_team_Delhi Daredevils  \
32                             0                          0   
33                             0                          0   
34                             0                          0   
35                             0                          0   
36                             0                          0   

    bat_team_Kings XI Punjab  ...  bat_team_Royal Challengers Bangalore  \
32                         0  ...                                     0   
33                         0  ...                                     0   
34                         0  ...                                     0   
35                         0  ...                                     0   
36                         0  ...                                     0   

    bat_team_Sunrisers Hyderabad  bowl_team_Chennai Super Kings  \
32                             0                              0   
33                             0                              0   
34                             0                              0   
35                             0                              0   
36                             0                              0   

    bowl_team_Delhi Daredevils  bowl_team_Kings XI Punjab  \
32                           0                          0   
33                           0                          0   
34                           0                          0   
35                           0                          0   
36                           0                          0   

    bowl_team_Kolkata Knight Riders  bowl_team_Mumbai Indians  \
32                                0                         0   
33                                0                         0   
34                                0                         0   
35                                0                         0   
36                                0                         0   

    bowl_team_Rajasthan Royals  bowl_team_Royal Challengers Bangalore  \
32                           0                                      1   
33                           0                                      1   
34                           0                                      1   
35                           0                                      1   
36                           0                                      1   

    bowl_team_Sunrisers Hyderabad  
32                              0  
33                              0  
34                              0  
35                              0  
36                              0  

[5 rows x 23 columns]

In [14]:
encoded_df.head()

date  runs  wickets  overs  runs_last_5  wickets_last_5  total  \
32 2008-04-18    61        0    5.1           59               0    222   
33 2008-04-18    61        1    5.2           59               1    222   
34 2008-04-18    61        1    5.3           59               1    222   
35 2008-04-18    61        1    5.4           59               1    222   
36 2008-04-18    61        1    5.5           58               1    222   

    bat_team_Chennai Super Kings  bat_team_Delhi Daredevils  \
32                             0                          0   
33                             0                          0   
34                             0                          0   
35                             0                          0   
36                             0                          0   

    bat_team_Kings XI Punjab  ...  bat_team_Royal Challengers Bangalore  \
32                         0  ...                                     0   
33                         0  ...                                     0   
34                         0  ...                                     0   
35                         0  ...                                     0   
36                         0  ...                                     0   

    bat_team_Sunrisers Hyderabad  bowl_team_Chennai Super Kings  \
32                             0                              0   
33                             0                              0   
34                             0                              0   
35                             0                              0   
36                             0                              0   

    bowl_team_Delhi Daredevils  bowl_team_Kings XI Punjab  \
32                           0                          0   
33                           0                          0   
34                           0                          0   
35                           0                          0   
36                           0                          0   

    bowl_team_Kolkata Knight Riders  bowl_team_Mumbai Indians  \
32                                0                         0   
33                                0                         0   
34                                0                         0   
35                                0                         0   
36                                0                         0   

    bowl_team_Rajasthan Royals  bowl_team_Royal Challengers Bangalore  \
32                           0                                      1   
33                           0                                      1   
34                           0                                      1   
35                           0                                      1   
36                           0                                      1   

    bowl_team_Sunrisers Hyderabad  
32                              0  
33                              0  
34                              0  
35                              0  
36                              0  

[5 rows x 23 columns]

In [16]:
encoded_df.columns

Index(['date', 'runs', 'wickets', 'overs', 'runs_last_5', 'wickets_last_5',
       'total', 'bat_team_Chennai Super Kings', 'bat_team_Delhi Daredevils',
       'bat_team_Kings XI Punjab', 'bat_team_Kolkata Knight Riders',
       'bat_team_Mumbai Indians', 'bat_team_Rajasthan Royals',
       'bat_team_Royal Challengers Bangalore', 'bat_team_Sunrisers Hyderabad',
       'bowl_team_Chennai Super Kings', 'bowl_team_Delhi Daredevils',
       'bowl_team_Kings XI Punjab', 'bowl_team_Kolkata Knight Riders',
       'bowl_team_Mumbai Indians', 'bowl_team_Rajasthan Royals',
       'bowl_team_Royal Challengers Bangalore',
       'bowl_team_Sunrisers Hyderabad'],
      dtype='object')

In [17]:
# Rearranging the columns
encoded_df = encoded_df[['date', 'bat_team_Chennai Super Kings', 'bat_team_Delhi Daredevils', 'bat_team_Kings XI Punjab',
              'bat_team_Kolkata Knight Riders', 'bat_team_Mumbai Indians', 'bat_team_Rajasthan Royals',
              'bat_team_Royal Challengers Bangalore', 'bat_team_Sunrisers Hyderabad',
              'bowl_team_Chennai Super Kings', 'bowl_team_Delhi Daredevils', 'bowl_team_Kings XI Punjab',
              'bowl_team_Kolkata Knight Riders', 'bowl_team_Mumbai Indians', 'bowl_team_Rajasthan Royals',
              'bowl_team_Royal Challengers Bangalore', 'bowl_team_Sunrisers Hyderabad',
              'overs', 'runs', 'wickets', 'runs_last_5', 'wickets_last_5', 'total']]

In [18]:
# Splitting the data into train and test set
X_train = encoded_df.drop(labels='total', axis=1)[encoded_df['date'].dt.year <= 2016]
X_test = encoded_df.drop(labels='total', axis=1)[encoded_df['date'].dt.year >= 2017]

In [19]:
y_train = encoded_df[encoded_df['date'].dt.year <= 2016]['total'].values
y_test = encoded_df[encoded_df['date'].dt.year >= 2017]['total'].values

In [20]:
# Removing the 'date' column
X_train.drop(labels='date', axis=True, inplace=True)
X_test.drop(labels='date', axis=True, inplace=True)

In [21]:
# --- Model Building ---
# Linear Regression Model
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [22]:
# Creating a pickle file for the classifier
filename = 'first-innings-score-lr-model.pkl'
pickle.dump(regressor, open(filename, 'wb'))

In [23]:
from sklearn.metrics import r2_score

In [25]:
score = r2_score(regressor.predict(X_test),y_test)
score

0.616929467383744

In [27]:
print(regressor.predict(X_test))

[172.00170898 175.1496582  174.54663086 ... 100.31518555  99.74609375
  93.08544922]


In [29]:
filename = 'finalized_IPL_model.pickle'
pickle.dump(regressor,open(filename,'wb'))